## Lenet 5 con MNIST

Charge Dataset

In [2]:
import torch
import torch.nn as nn
import numpy as np
from joblib import Parallel, delayed
import torch.nn.functional as F

In [3]:
#Import data
from torchvision import datasets
import torchvision.transforms as transforms
batch_size = 1000
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST(root='./data', train=False, download=True,
                   transform=transforms.ToTensor()),
    batch_size=batch_size, shuffle=False)

for data, label in test_loader:
    break
data, label = data, label

100%|██████████| 9912422/9912422 [00:00<00:00, 378177603.94it/s]

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


100%|██████████| 28881/28881 [00:00<00:00, 17042162.89it/s]


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



100%|██████████| 1648877/1648877 [00:00<00:00, 220539283.67it/s]

Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw




100%|██████████| 4542/4542 [00:00<00:00, 18406307.99it/s]


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw



Exact model

In [4]:
class exact_model(nn.Module):

    def __init__(self):
        super(exact_model, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1, stride=1)
        self.relu1 = nn.ReLU(inplace=True)
        self.maxpool1 = nn.MaxPool2d(2)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1, stride=1)
        self.relu2 = nn.ReLU(inplace=True)
        self.maxpool2 = nn.MaxPool2d(2)
        self.linear1 = nn.Linear(7 * 7 * 64, 200)
        self.relu3 = nn.ReLU(inplace=True)
        self.linear2 = nn.Linear(200, 10)

    def forward(self, x):
        x = self.maxpool1(self.relu1(self.conv1(x)))
        x = self.maxpool2(self.relu2(self.conv2(x)))
        x = x.view(x.size(0), -1)
        x = self.relu3(self.linear1(x))
        x = self.linear2(x)
        #x = F.softmax(x, dim = 1)
        return x

model_exact = exact_model()
filename = "trained_lenet5.pt"

model_exact.load_state_dict(torch.load(filename, map_location=torch.device('cpu') ))
model_exact.eval()

exact_model(
  (conv1): Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu1): ReLU(inplace=True)
  (maxpool1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (relu2): ReLU(inplace=True)
  (maxpool2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (linear1): Linear(in_features=3136, out_features=200, bias=True)
  (relu3): ReLU(inplace=True)
  (linear2): Linear(in_features=200, out_features=10, bias=True)
)

Prediction

In [6]:
i =  0
x = data[i].unsqueeze(0)
y = label[i].unsqueeze(0)

scores_exact = model_exact(x)
pred_exact = model_exact(x).data.max(1, keepdim=True)[1][0].item()
print("Scores exact:", scores_exact)
print("Prediction exact:", pred_exact)
print("Label:", y)

Scores exact: tensor([[ -4.9248,  -1.6787,   1.7533,   2.3358,  -7.6406,  -3.2882, -22.0243,
          14.6021,  -1.9173,   2.2584]], grad_fn=<AddmmBackward0>)
Prediction exact: 7
Label: tensor([7])
